<h1><center>Ind Vs Pakistan Cricket Hackathon

In [142]:
#importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pycaret.regression import *
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.linear_model import BayesianRidge, Ridge
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor

In [2]:
# Reading the dataset into pandas DataFrame
df = pd.read_csv('train-file.csv')
df.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id
0,1,Rohit Sharma,DNB,-,-,1,0,23 Jun 2007,v Ireland Belfast,1
1,1,Rohit Sharma,8,0,3,1,0,26 Jun 2007,v South Africa Belfast,2
2,1,Rohit Sharma,1,-,-,0,0,5 Oct 2007,v Australia Hyderabad,3
3,1,Rohit Sharma,52,-,-,1,0,18 Nov 2007,v Pakistan Jaipur,4
4,1,Rohit Sharma,29,-,-,0,0,3 Feb 2008,v Australia Brisbane,5


In [3]:
df.shape

(2021, 10)

In [4]:
#Understanding the nature of each feature
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_id      2021 non-null   int64 
 1   player_name    2021 non-null   object
 2   runs_scored    2021 non-null   object
 3   wickets        2021 non-null   object
 4   runs_conceded  2021 non-null   object
 5   catches        2021 non-null   object
 6   stumpings      2021 non-null   object
 7   match_date     2021 non-null   object
 8   opposition     2021 non-null   object
 9   match_id       2021 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 158.0+ KB


<b>Here the runs_scored, wickets, runs_conceded, catches and stumpings are to be numerical features. The opposition has two data, one is the team against which India played and the location of the match. Feature transformation has to be done on this feature.

In [5]:
#Knowing each column contents to check for data inconsistencies
df.player_name.value_counts()

Virat Kohli            281
Rohit Sharma           251
Ravindra Jadeja        186
Ravichandran ashwin    115
Babar Azam             108
Mohammed Shami          94
Kuldeep Yadav           90
Hardik Pandya           82
Faqar Zaman             78
Jasprit Bumrah          78
Imam-Ul-Haq             66
Mohammad Rizwan         65
Shadab Khan             64
KL Rahul                61
Shreyas Iyer            47
Shardul Thakur          44
Shaheen Shah Afridi     44
Shubman Gill            35
Mohammad Nawaz          32
Suryakumar Yadav        30
Mohammed Siraj          30
Haris Rauf              28
Ishan Kishan            25
Iftikhar Ahmed          19
Salman Agha             18
Hasan Ali               16
Mohammad Wasim Jr.      16
Usama Mir                8
Saud Shakeel             6
Abdullah Shafique        4
Name: player_name, dtype: int64

In [6]:
print('Total Number of players in this dataset are', len(df.player_name.unique()))

Total Number of players in this dataset are 30


<b>This dataset has both Indian and Pakistani players.

In [7]:
df.runs_scored.unique()

array(['DNB', '8', '1', '52', '29', '0', '39*', '70*', '24', '3*', '66',
       '2', '9', '26', '11', '0*', '22', '58', '22*', '3', '19', '12',
       '37', '32', '25', '18', '54', '31', '11*', '28', '8*', '25*', '4*',
       '15', '60*', '43*', '4', '7', 'TDNB', '48', '57', '36', '114',
       '61*', '101*', '82', '13', '51', '68', '38', '6', '69', '10', '41',
       '20', '5', '44', '23', '87*', '68*', '7*', '81', '86*', '39', '94',
       '55', '72', '90*', '117', '9*', '95', '27', '23*', '21', '80',
       '77', '24*', '30*', '33', '1*', '14', '108', '183', '106', '128*',
       '2*', '31*', '83', '21*', '65', '47*', '58*', '43', '33*', '60',
       '5*', '49*', '46', '102', '48*', '115', '6*', '64*', '42', '61',
       '141*', '100*', '79', '115*', '209', '86', '99', '123', '78',
       '66*', '62*', '14*', '136', '56', '52*', '18*', '10*', '264',
       '139*', '138', '107', '57*', '16*', '64', '44*', '16', '137', '34',
       '63', '150', '171*', '91', '124', '59', '85*', '154*'

In [8]:
df.wickets.unique()

array(['-', '0', '2', '1', '3', '5', '4', '6'], dtype=object)

In [9]:
df.runs_conceded.unique()

array(['-', '3', '5', '4', '11', '9', '13', '7', '22', '40', '34', '27',
       '12', '28', '53', '51', '49', '54', '50', '43', '29', '16', '63',
       '19', '66', '24', '30', '6', '14', '39', '8', '42', '74', '37',
       '33', '59', '62', '17', '41', '48', '32', '15', '47', '10', '58',
       '31', '56', '46', '18', '23', '21', '26', '36', '35', '44', '55',
       '68', '67', '25', '20', '45', '52', '72', '57', '64', '73', '69',
       '61', '84', '81', '38', '71', '75', '1', '60', '70', '2', '76',
       '65', '77', '79', '78', '80', '87', '0', '83', '82'], dtype=object)

In [10]:
df.catches.unique()

array(['1', '0', '-', '3', '2', '4'], dtype=object)

In [11]:
df.stumpings.unique()

array(['0', '-', '1'], dtype=object)

<b>Here, it would be optimal to convert DNB to 0, and since the problem concentrates on how much would a person score, we can remove the * from scores.

In [12]:
for i in ['runs_scored', 'runs_conceded', 'wickets', 'catches','stumpings']:
    for j in range(len(df[i])):
        df[i][j] = str(df[i][j]).replace('TDNB','0')
        df[i][j] = str(df[i][j]).replace('DNB','0')
        df[i][j] = str(df[i][j]).replace('absent','0')
        df[i][j] = str(df[i][j]).replace('*','')
        df[i][j] = str(df[i][j]).replace('-','0')

In [13]:
#Changing the data type of the features
df = df.astype({'runs_scored':'int64','runs_conceded':'int64','wickets':'int64','catches':'int64','stumpings':'int64'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_id      2021 non-null   int64 
 1   player_name    2021 non-null   object
 2   runs_scored    2021 non-null   int64 
 3   wickets        2021 non-null   int64 
 4   runs_conceded  2021 non-null   int64 
 5   catches        2021 non-null   int64 
 6   stumpings      2021 non-null   int64 
 7   match_date     2021 non-null   object
 8   opposition     2021 non-null   object
 9   match_id       2021 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 158.0+ KB


In [14]:
df.head(2)

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id
0,1,Rohit Sharma,0,0,0,1,0,23 Jun 2007,v Ireland Belfast,1
1,1,Rohit Sharma,8,0,3,1,0,26 Jun 2007,v South Africa Belfast,2


<b>match_id and match_date are providing the same meaning and therefore match_date can be dropped from the df

In [15]:
df.drop(columns = 'match_date',inplace=True)

In [16]:
df.opposition.unique()

array(['v Ireland Belfast', 'v South Africa Belfast',
       'v Australia Hyderabad', 'v Pakistan Jaipur',
       'v Australia Brisbane', 'v Sri Lanka Brisbane',
       'v Australia Melbourne', 'v Sri Lanka Canberra',
       'v Australia Adelaide', 'v Sri Lanka Adelaide',
       'v Australia Sydney', 'v Sri Lanka Hobart', 'v Pakistan Mirpur',
       'v Bangladesh Mirpur', 'v Hong Kong Karachi', 'v Pakistan Karachi',
       'v Bangladesh Karachi', 'v Sri Lanka Karachi',
       'v Sri Lanka Dambulla', 'v Sri Lanka Colombo (RPS)',
       'v England Rajkot', 'v England Indore', 'v England Kanpur',
       'v England Cuttack', 'v New Zealand Hamilton',
       'v New Zealand Auckland', 'v West Indies Kingston',
       'v West Indies Gros Islet', 'v South Africa Ahmedabad',
       'v Zimbabwe Bulawayo', 'v Sri Lanka Bulawayo', 'v Zimbabwe Harare',
       'v Sri Lanka Harare', 'v Bangladesh Dambulla',
       'v Pakistan Dambulla', 'v New Zealand Dambulla',
       'v New Zealand Bengaluru', 'v N

In [17]:
international_cricket_teams = ["Australia", "Bangladesh", "England", "India", "New Zealand", "Pakistan",
                               "South Africa", "Sri Lanka", "West Indies", "Zimbabwe","Ireland",
                              "Hong Kong", "Afghanistan", "Nepal","Netherlands","U.A.E."]
df['opposition_team'] = ''
for i in range(len(df.opposition)):
    for j in international_cricket_teams:
        if j in df.opposition[i]:
            df['opposition_team'][i] = j

In [18]:
df.opposition_team.unique()

array(['Ireland', 'South Africa', 'Australia', 'Pakistan', 'Sri Lanka',
       'Bangladesh', 'Hong Kong', 'England', 'New Zealand', 'West Indies',
       'Zimbabwe', 'Afghanistan', 'U.A.E.', 'India', 'Nepal',
       'Netherlands'], dtype=object)

In [19]:
pattern = 'Geeks f'
string = 'Geeks for geeks'
last_index = (re.search(pattern, string)).span()[1]

In [20]:
string[last_index:]

'or geeks'

In [21]:
df['Location'] = ''
for i in range(df.shape[0]):
    index_end_loc = (re.search(df['opposition_team'][i],df['opposition'][i])).span()[1]
    df['Location'][i] = df['opposition'][i][index_end_loc + 1:]

In [22]:
df.head(3)

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,opposition,match_id,opposition_team,Location
0,1,Rohit Sharma,0,0,0,1,0,v Ireland Belfast,1,Ireland,Belfast
1,1,Rohit Sharma,8,0,3,1,0,v South Africa Belfast,2,South Africa,Belfast
2,1,Rohit Sharma,1,0,0,0,0,v Australia Hyderabad,3,Australia,Hyderabad


In [23]:
df.Location.unique()

array(['Belfast', 'Hyderabad', 'Jaipur', 'Brisbane', 'Melbourne',
       'Canberra', 'Adelaide', 'Sydney', 'Hobart', 'Mirpur', 'Karachi',
       'Dambulla', 'Colombo (RPS)', 'Rajkot', 'Indore', 'Kanpur',
       'Cuttack', 'Hamilton', 'Auckland', 'Kingston', 'Gros Islet',
       'Ahmedabad', 'Bulawayo', 'Harare', 'Bengaluru', 'Chennai',
       'Durban', 'Johannesburg', 'Cape Town', 'Gqeberha', 'Centurion',
       'Port of Spain', 'North Sound', 'Chester-le-Street',
       'Visakhapatnam', 'Perth', 'Hambantota', 'Pallekele', 'Mohali',
       'Dharamsala', 'Cardiff', 'The Oval', 'Birmingham', 'Pune',
       'Ranchi', 'Nagpur', 'Kochi', 'Napier', 'Wellington', 'Fatullah',
       'Eden Gardens', 'Wankhede', 'Delhi', 'Nottingham', "Lord's",
       'Leeds', 'Dubai (DSC)', 'Guwahati', 'Brabourne',
       'Thiruvananthapuram', 'Mount Maunganui', 'Southampton',
       'Manchester', 'Providence', 'Raipur', 'Bridgetown', 'Tarouba',
       'Lucknow', 'Christchurch', 'Vadodara', 'Gwalior', 'Paarl',


In [24]:
indian_cities_with_international_cricket_grounds = ["Mumbai", "Kolkata", "Chennai", "Bengaluru", "Delhi", 
                                                    "Hyderabad", "Mohali", "Ahmedabad", "Rajkot", "Kanpur", 
                                                    "Nagpur", "Pune", "Indore", "Jaipur", "Dharamshala", 
                                                    "Thiruvananthapuram", "Guwahati", "Visakhapatnam", "Ranchi", 
                                                    "Lucknow", "Noida", "Cuttack", "Kochi", "Guwahati", "Trichy",
                                                   "Wankhede",'Raipur','Vadodara', 'Gwalior']
df['match_location'] = ''
for i in range(len(df.Location)):
    if df.Location.iloc[i] in indian_cities_with_international_cricket_grounds:
        df['match_location'].iloc[i] = 'Home'
    else:
        df['match_location'].iloc[i] = 'Away'


In [25]:
(df[df.match_location == 'Away']).Location.unique()

array(['Belfast', 'Brisbane', 'Melbourne', 'Canberra', 'Adelaide',
       'Sydney', 'Hobart', 'Mirpur', 'Karachi', 'Dambulla',
       'Colombo (RPS)', 'Hamilton', 'Auckland', 'Kingston', 'Gros Islet',
       'Bulawayo', 'Harare', 'Durban', 'Johannesburg', 'Cape Town',
       'Gqeberha', 'Centurion', 'Port of Spain', 'North Sound',
       'Chester-le-Street', 'Perth', 'Hambantota', 'Pallekele',
       'Dharamsala', 'Cardiff', 'The Oval', 'Birmingham', 'Napier',
       'Wellington', 'Fatullah', 'Eden Gardens', 'Nottingham', "Lord's",
       'Leeds', 'Dubai (DSC)', 'Brabourne', 'Mount Maunganui',
       'Southampton', 'Manchester', 'Providence', 'Bridgetown', 'Tarouba',
       'Christchurch', 'Paarl', 'Chattogram', 'Lahore', 'Abu Dhabi',
       'Sharjah', 'Dublin (Malahide)', 'Nelson', 'Dunedin', 'Bristol',
       'Taunton', 'Rawalpindi', 'Multan', 'Rotterdam'], dtype=object)

In [26]:
df.columns

Index(['player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded',
       'catches', 'stumpings', 'opposition', 'match_id', 'opposition_team',
       'Location', 'match_location'],
      dtype='object')

In [27]:
df.corr()

,player_id,runs_scored,wickets,runs_conceded,catches,stumpings,match_id
player_id,1.000000,-0.158350,0.162327,0.178207,0.003962,0.029348,0.730379
runs_scored,-0.158350,1.000000,-0.321344,-0.400904,0.079816,0.064208,0.023148
wickets,0.162327,-0.321344,1.000000,0.558357,-0.091126,-0.040091,0.018036
runs_conceded,0.178207,-0.400904,0.558357,1.000000,-0.140269,-0.057623,-0.026254
catches,0.003962,0.079816,-0.091126,-0.140269,1.000000,0.051594,0.031140
stumpings,0.029348,0.064208,-0.040091,-0.057623,0.051594,1.000000,0.040592
match_id,0.730379,0.023148,0.018036,-0.026254,0.031140,0.040592,1.000000


In [28]:
df1 = pd.get_dummies(data=df,columns=['opposition_team','match_location'],drop_first=True)

In [29]:
df1.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,opposition,match_id,Location,...,opposition_team_Nepal,opposition_team_Netherlands,opposition_team_New Zealand,opposition_team_Pakistan,opposition_team_South Africa,opposition_team_Sri Lanka,opposition_team_U.A.E.,opposition_team_West Indies,opposition_team_Zimbabwe,match_location_Home
0,1,Rohit Sharma,0,0,0,1,0,v Ireland Belfast,1,Belfast,...,0,0,0,0,0,0,0,0,0,0
1,1,Rohit Sharma,8,0,3,1,0,v South Africa Belfast,2,Belfast,...,0,0,0,0,1,0,0,0,0,0
2,1,Rohit Sharma,1,0,0,0,0,v Australia Hyderabad,3,Hyderabad,...,0,0,0,0,0,0,0,0,0,1
3,1,Rohit Sharma,52,0,0,1,0,v Pakistan Jaipur,4,Jaipur,...,0,0,0,1,0,0,0,0,0,1
4,1,Rohit Sharma,29,0,0,0,0,v Australia Brisbane,5,Brisbane,...,0,0,0,0,0,0,0,0,0,0


In [30]:
player_dict = df[['player_id','player_name']].drop_duplicates().set_index('player_id').to_dict()
print(player_dict)

{'player_name': {1: 'Rohit Sharma', 4: 'Virat Kohli', 7: 'Ravindra Jadeja', 13: 'Ravichandran ashwin', 12: 'Mohammed Shami', 9: 'Jasprit Bumrah', 2: 'Hardik Pandya', 16: 'Babar Azam', 17: 'Shadab Khan', 19: 'Faqar Zaman', 6: 'KL Rahul', 8: 'Shardul Thakur', 11: 'Kuldeep Yadav', 5: 'Shreyas Iyer', 23: 'Imam-Ul-Haq', 24: 'Mohammad Nawaz', 29: 'Shaheen Shah Afridi', 10: 'Mohammed Siraj', 3: 'Shubman Gill', 14: 'Ishan Kishan', 15: 'Suryakumar Yadav', 20: 'Haris Rauf', 22: 'Iftikhar Ahmed', 25: 'Mohammad Rizwan', 27: 'Salman Agha', 28: 'Saud Shakeel', 21: 'Hasan Ali', 26: 'Mohammad Wasim Jr.', 18: 'Abdullah Shafique', 30: 'Usama Mir'}}


In [31]:
df1.drop(columns={'player_name','opposition','runs_conceded','catches','stumpings','Location'},inplace=True)

In [32]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   player_id                     2021 non-null   int64
 1   runs_scored                   2021 non-null   int64
 2   wickets                       2021 non-null   int64
 3   match_id                      2021 non-null   int64
 4   opposition_team_Australia     2021 non-null   uint8
 5   opposition_team_Bangladesh    2021 non-null   uint8
 6   opposition_team_England       2021 non-null   uint8
 7   opposition_team_Hong Kong     2021 non-null   uint8
 8   opposition_team_India         2021 non-null   uint8
 9   opposition_team_Ireland       2021 non-null   uint8
 10  opposition_team_Nepal         2021 non-null   uint8
 11  opposition_team_Netherlands   2021 non-null   uint8
 12  opposition_team_New Zealand   2021 non-null   uint8
 13  opposition_team_Pakistan      202

In [91]:
X_runs = df1.drop(columns = {'runs_scored'})
y_runs = np.log1p(df1[['runs_scored']])

In [122]:
X_train,X_test,y_train,y_test = train_test_split(X_runs,y_runs,test_size=0.2,random_state=42)

In [123]:
runs_train_data_combined = X_train.copy()
runs_train_data_combined['runs_scored'] = y_train

In [124]:
fit_data = setup(data = runs_train_data_combined,target = 'runs_scored')
best = compare_models()

,Description,Value
0,Session id,3706
1,Target,runs_scored
2,Target type,Regression
3,Original data shape,"(1616, 20)"
4,Transformed data shape,"(1616, 20)"
5,Transformed train set shape,"(1131, 20)"
6,Transformed test set shape,"(485, 20)"
7,Numeric features,19
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,1.1431,1.9434,1.3917,0.3174,0.5677,0.4226,0.0170
ada,AdaBoost Regressor,1.2091,1.9888,1.4088,0.3032,0.5836,0.4129,0.0080
lightgbm,Light Gradient Boosting Machine,1.1424,2.0352,1.4240,0.2848,0.5686,0.4443,0.2840
rf,Random Forest Regressor,1.1972,2.3006,1.5144,0.1917,0.5979,0.4651,0.0390
br,Bayesian Ridge,1.2978,2.3142,1.5203,0.1884,0.6328,0.4412,0.0040
ridge,Ridge Regression,1.2932,2.3309,1.5259,0.1823,0.6322,0.4426,0.0050
lr,Linear Regression,1.2939,2.3385,1.5283,0.1797,0.6327,0.4431,0.0100
huber,Huber Regressor,1.2901,2.3556,1.5336,0.1726,0.6352,0.4521,0.0080
knn,K Neighbors Regressor,1.2592,2.4022,1.5480,0.1575,0.6192,0.4475,0.0040
omp,Orthogonal Matching Pursuit,1.3322,2.4294,1.5578,0.1499,0.6494,0.4475,0.0050


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

Let us create an esemble model of the top 5 regressors by assigning weights to each one of them, in the order of reducing ranks.

In [125]:
models_runs = {
    'ada' : AdaBoostRegressor(),
    'Gradient_boosting_reg' : GradientBoostingRegressor(),
    'Bayesian_Reidge' : BayesianRidge(),
    'Ridge_Regression' : Ridge(),
    'Light_Gradient_Boosting' : LGBMRegressor(),
    'Random_forest_regression' : RandomForestRegressor() 
}

In [126]:
for name, model in models_runs.items():
    model.fit(X_train,y_train)
    print(name,"is trained")

ada is trained
Gradient_boosting_reg is trained
Bayesian_Reidge is trained
Ridge_Regression is trained
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002246 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 316
[LightGBM] [Info] Number of data points in the train set: 1616, number of used features: 15
[LightGBM] [Info] Start training from score 2.165225
Light_Gradient_Boosting is trained
Random_forest_regression is trained


In [127]:
for model_name,model in models_runs.items():
    model_result = np.exp(np.sqrt(-cross_val_score(estimator = model,X=X_train,y=y_train,cv=5, 
                                   scoring = 'neg_root_mean_squared_error')))
    print(model_name,'\n')
    print('Mean: ',model_result.mean(),'\n')
    print('Std: ',model_result.std(),'\n')

ada 

Mean:  3.299494218648666 

Std:  0.06761502206896308 

Gradient_boosting_reg 

Mean:  3.2580209243973526 

Std:  0.09142170975280715 

Bayesian_Reidge 

Mean:  3.440613575820315 

Std:  0.0586543755861372 

Ridge_Regression 

Mean:  3.447339753038019 

Std:  0.06391471237638373 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001176 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 1292, number of used features: 15
[LightGBM] [Info] Start training from score 2.156289
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000342 seconds.
You can set `force_row_wise=true` to remove t

In [128]:
y_pred_runs = np.around(np.exp(
        0.3 * models_runs['ada'].predict(X_test) + 
        0.3 * models_runs['Gradient_boosting_reg'].predict(X_test) + 
        0.2 * models_runs['Light_Gradient_Boosting'].predict(X_test) + 
        0.1 * models_runs['Random_forest_regression'].predict(X_test) + 
        0.1 * models_runs['Bayesian_Reidge'].predict(X_test))+1,decimals=0)

In [144]:
y_pred_runs = np.log1p(y_pred_runs)

In [145]:
mean_squared_error(y_test,y_pred_runs)

5.742414918998356

<h3> Now building model to predict wickets secured by each player

In [130]:
X_wickets = df1.drop(columns = {'wickets'})
y_wickets = df1[['wickets']]

In [131]:
X_train,X_test,y_train,y_test = train_test_split(X_wickets,y_wickets,test_size=0.2,random_state=42)

In [132]:
wickets_train_data_combined = X_train.copy()
wickets_train_data_combined['wickets'] = y_train
fit_data = setup(data = wickets_train_data_combined,target = 'wickets')
best = compare_models()

,Description,Value
0,Session id,5659
1,Target,wickets
2,Target type,Regression
3,Original data shape,"(1616, 20)"
4,Transformed data shape,"(1616, 20)"
5,Transformed train set shape,"(1131, 20)"
6,Transformed test set shape,"(485, 20)"
7,Numeric features,19
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.5604,0.8288,0.9047,0.3320,0.4057,0.4799,0.0180
lightgbm,Light Gradient Boosting Machine,0.6046,0.9162,0.9513,0.2585,0.4296,0.5139,0.2280
ada,AdaBoost Regressor,0.7089,0.9867,0.9889,0.1992,0.4961,0.4490,0.0080
rf,Random Forest Regressor,0.6026,1.0137,1.0014,0.1788,0.4463,0.5607,0.0360
br,Bayesian Ridge,0.7936,1.1043,1.0461,0.1174,0.5163,0.4700,0.0040
ridge,Ridge Regression,0.7937,1.1071,1.0477,0.1133,0.5179,0.4816,0.0040
lr,Linear Regression,0.7957,1.1130,1.0506,0.1084,0.5195,0.4833,0.0070
en,Elastic Net,0.8056,1.1310,1.0582,0.0977,0.5219,0.4651,0.0040
omp,Orthogonal Matching Pursuit,0.8092,1.1368,1.0609,0.0931,0.5242,0.4612,0.0040
lasso,Lasso Regression,0.8087,1.1379,1.0614,0.0922,0.5229,0.4712,0.0050


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [133]:
models_wickets = {
    'ada' : AdaBoostRegressor(),
    'Gradient_boosting_reg' : GradientBoostingRegressor(),
    'Bayesian_Reidge' : BayesianRidge(),
    'Ridge_Regression' : Ridge(),
    'Light_Gradient_Boosting' : LGBMRegressor(),
    'Random_forest_regression' : RandomForestRegressor() 
}

In [134]:
for name, model in models_wickets.items():
    model.fit(X_train,y_train)
    print(name,"is trained")

ada is trained
Gradient_boosting_reg is trained
Bayesian_Reidge is trained
Ridge_Regression is trained
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000432 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 429
[LightGBM] [Info] Number of data points in the train set: 1616, number of used features: 15
[LightGBM] [Info] Start training from score 0.666460
Light_Gradient_Boosting is trained
Random_forest_regression is trained


In [135]:
for model_name,model in models_wickets.items():
    model_result = np.exp(np.sqrt(-cross_val_score(estimator = model,X=X_train,y=y_train,cv=5, 
                                   scoring = 'neg_root_mean_squared_error')))
    print(model_name,'\n')
    print('Mean: ',model_result.mean(),'\n')
    print('Std: ',model_result.std(),'\n')

ada 

Mean:  2.7525545408925676 

Std:  0.058854757989726815 

Gradient_boosting_reg 

Mean:  2.6025760353954017 

Std:  0.08570013044656122 

Bayesian_Reidge 

Mean:  2.79522117455497 

Std:  0.04792201670386804 

Ridge_Regression 

Mean:  2.7943643427612406 

Std:  0.04920680955955656 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 412
[LightGBM] [Info] Number of data points in the train set: 1292, number of used features: 15
[LightGBM] [Info] Start training from score 0.666409
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000180 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 413
[LightGBM] [Inf

In [139]:
y_pred_wickets = np.around(
        0.2 * models_wickets['ada'].predict(X_test) + 
        0.3 * models_wickets['Gradient_boosting_reg'].predict(X_test) + 
        0.3 * models_wickets['Light_Gradient_Boosting'].predict(X_test) + 
        0.1 * models_wickets['Random_forest_regression'].predict(X_test) + 
        0.1 * models_wickets['Bayesian_Reidge'].predict(X_test),decimals=0)

In [141]:
y_pred_wickets[:10]

array([2., 1., 1., 0., 1., 0., 1., 0., 0., 0.])

In [50]:
result_data = pd.DataFrame(columns=X_test.columns)

In [51]:
result_data.player_id = [i for i in range(1,31)]
result_data.head()

,player_id,match_id,opposition_team_Australia,opposition_team_Bangladesh,opposition_team_England,opposition_team_Hong Kong,opposition_team_India,opposition_team_Ireland,opposition_team_Nepal,opposition_team_Netherlands,opposition_team_New Zealand,opposition_team_Pakistan,opposition_team_South Africa,opposition_team_Sri Lanka,opposition_team_U.A.E.,opposition_team_West Indies,opposition_team_Zimbabwe,match_location_Home
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
result_data.columns

Index(['player_id', 'match_id', 'opposition_team_Australia',
       'opposition_team_Bangladesh', 'opposition_team_England',
       'opposition_team_Hong Kong', 'opposition_team_India',
       'opposition_team_Ireland', 'opposition_team_Nepal',
       'opposition_team_Netherlands', 'opposition_team_New Zealand',
       'opposition_team_Pakistan', 'opposition_team_South Africa',
       'opposition_team_Sri Lanka', 'opposition_team_U.A.E.',
       'opposition_team_West Indies', 'opposition_team_Zimbabwe',
       'match_location_Home'],
      dtype='object')

In [53]:
result_data.match_id = X.match_id.max() + 1
columns_0 = ['opposition_team_Australia',
       'opposition_team_Bangladesh', 'opposition_team_England',
       'opposition_team_Hong Kong', 'opposition_team_India',
       'opposition_team_Ireland', 'opposition_team_Nepal',
       'opposition_team_Netherlands', 'opposition_team_New Zealand', 'opposition_team_South Africa',
       'opposition_team_Sri Lanka', 'opposition_team_U.A.E.',
       'opposition_team_West Indies', 'opposition_team_Zimbabwe']
for i in columns_0:
    result_data[i] = 0
result_data.opposition_team_Pakistan = 1
result_data.head()

,player_id,match_id,opposition_team_Australia,opposition_team_Bangladesh,opposition_team_England,opposition_team_Hong Kong,opposition_team_India,opposition_team_Ireland,opposition_team_Nepal,opposition_team_Netherlands,opposition_team_New Zealand,opposition_team_Pakistan,opposition_team_South Africa,opposition_team_Sri Lanka,opposition_team_U.A.E.,opposition_team_West Indies,opposition_team_Zimbabwe,match_location_Home
0,1,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
1,2,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
2,3,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
3,4,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
4,5,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN


In [54]:
pred = rf.predict(result_data)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
for i in range(len(pred)):
    for j in range(0,2):
        pred[i][j] = int(round(pred[i][j]))

In [ ]:
final_results = pd.DataFrame(pred)
final_results.head()

In [ ]:
final_results['player_id'] = [i for i in range(1,31)]
final_results.head()

In [ ]:
final_results.rename(columns={0:'runs',1:'wickets'},inplace = True)

In [ ]:
final_results.head()

In [ ]:
final_results = final_results.iloc[:,[2,0,1]]
final_results.head()

In [ ]:
final_results.to_csv('submission1.csv',index = False)